In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm


In [106]:
IMG_DIR = "/content/images"
CSV_PATH = "/content/10_Labels.csv"

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
dataset = AdversarialImageDataset(
    img_dir=IMG_DIR,
    csv_path=CSV_PATH,
    transform=transform
)

print("Total samples:", len(dataset))


Total samples: 3925


In [ ]:
from PIL import Image
import os

folder_path = '/content/images/'

for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(folder_path, filename)
        with Image.open(img_path) as img:
            width, height = img.size
            print(f"{filename}: {width}x{height}")

 121.jpg: 231x231
 86.jpg: 231x231
 105.jpg: 231x231
 51.jpg: 231x231
 125.jpg: 231x231
 57.jpg: 231x231
 14.jpg: 231x231
 52.jpg: 231x231
 36.jpg: 231x231
 48.jpg: 231x231
 50.jpg: 231x231
 12.jpg: 231x231
 133.jpg: 231x231
 76.jpg: 231x231
 59.jpg: 231x231
 78.jpg: 231x231
 17.jpg: 231x231
 150.jpg: 231x231
 130.jpg: 231x231
 35.jpg: 231x231
 91.jpg: 231x231
 27.jpg: 231x231
 89.jpg: 231x231
 143.jpg: 231x231
 103.jpg: 231x231
 8.jpg: 231x231
 147.jpg: 231x231
 40.jpg: 231x231
 123.jpg: 231x231
 127.jpg: 231x231
 38.jpg: 231x231
 20.jpg: 231x231
 58.jpg: 231x231
 18.jpg: 231x231
 136.jpg: 231x231
 94.jpg: 231x231
 104.jpg: 231x231
 128.jpg: 231x231
 156.jpg: 231x231
 148.jpg: 231x231
 152.jpg: 231x231
 107.jpg: 231x231
 149.jpg: 231x231
 129.jpg: 231x231
 84.jpg: 231x231
 154.jpg: 231x231
 146.jpg: 231x231
 142.jpg: 231x231
 110.jpg: 231x231
 80.jpg: 231x231
 139.jpg: 231x231
 61.jpg: 231x231
 85.jpg: 231x231
 79.jpg: 231x231
 73.jpg: 231x231
 71.jpg: 231x231
 157.jpg: 231x231
 132.j

In [94]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os
from torchvision import transforms

class AdversarialImageDataset(Dataset):
    def __init__(self, csv_path, image_dir):
        self.data = pd.read_csv(csv_path)
        self.image_dir = image_dir

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),   # handles different image sizes
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # ✅ CSV already contains correct filename (e.g. "2980.jpg")
        img_name = str(row["image"]).strip()

        img_path = os.path.join(self.image_dir, img_name)

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        label = int(row["adv"])  # 0 = clean, 1 = adversarial

        return image, label


In [99]:
dataset = AdversarialImageDataset(
    csv_path="/content/10_Labels.csv",
    image_dir="/content/images"
)

img, label = dataset[0]
print("Image shape:", img.shape)
print("Label:", label)


Image shape: torch.Size([3, 224, 224])
Label: 0


In [97]:
# Check disk
print("Files on disk (first 5):", os.listdir("/content/images")[:5])

# Check CSV
df = pd.read_csv("/content/10_Labels.csv")
print("Files in CSV (first 5):", df["image"].head().tolist())

Files on disk (first 5): [' 121.jpg', ' 86.jpg', ' 105.jpg', ' 51.jpg', ' 125.jpg']
Files in CSV (first 5): ['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg']


In [98]:
import os

image_dir = "/content/images"

for filename in os.listdir(image_dir):
    # .strip() removes spaces from the beginning and end
    clean_name = filename.strip()

    if clean_name != filename:
        old_path = os.path.join(image_dir, filename)
        new_path = os.path.join(image_dir, clean_name)

        # Rename the file on disk
        os.rename(old_path, new_path)
        print(f"Fixed: '{filename}' -> '{clean_name}'")

print("\nCleanup complete. Try running your Dataset code again!")

Fixed: ' 121.jpg' -> '121.jpg'
Fixed: ' 86.jpg' -> '86.jpg'
Fixed: ' 105.jpg' -> '105.jpg'
Fixed: ' 51.jpg' -> '51.jpg'
Fixed: ' 125.jpg' -> '125.jpg'
Fixed: ' 57.jpg' -> '57.jpg'
Fixed: ' 14.jpg' -> '14.jpg'
Fixed: ' 52.jpg' -> '52.jpg'
Fixed: ' 36.jpg' -> '36.jpg'
Fixed: ' 48.jpg' -> '48.jpg'
Fixed: ' 50.jpg' -> '50.jpg'
Fixed: ' 12.jpg' -> '12.jpg'
Fixed: ' 133.jpg' -> '133.jpg'
Fixed: ' 76.jpg' -> '76.jpg'
Fixed: ' 59.jpg' -> '59.jpg'
Fixed: ' 78.jpg' -> '78.jpg'
Fixed: ' 17.jpg' -> '17.jpg'
Fixed: ' 150.jpg' -> '150.jpg'
Fixed: ' 130.jpg' -> '130.jpg'
Fixed: ' 35.jpg' -> '35.jpg'
Fixed: ' 91.jpg' -> '91.jpg'
Fixed: ' 27.jpg' -> '27.jpg'
Fixed: ' 89.jpg' -> '89.jpg'
Fixed: ' 143.jpg' -> '143.jpg'
Fixed: ' 103.jpg' -> '103.jpg'
Fixed: ' 8.jpg' -> '8.jpg'
Fixed: ' 147.jpg' -> '147.jpg'
Fixed: ' 40.jpg' -> '40.jpg'
Fixed: ' 123.jpg' -> '123.jpg'
Fixed: ' 127.jpg' -> '127.jpg'
Fixed: ' 38.jpg' -> '38.jpg'
Fixed: ' 20.jpg' -> '20.jpg'
Fixed: ' 58.jpg' -> '58.jpg'
Fixed: ' 18.jpg' -> '18

In [104]:
import os
import re

image_dir = "/content/images"

# 1. Get all files and clean hidden spaces/parentheses first
files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# 2. Sort files numerically
# (This ensures 2.jpg comes before 10.jpg)
def extract_number(f):
    s = re.findall(r'\d+', f)
    return int(s[0]) if s else 0

files.sort(key=extract_number)

# 3. Rename them to match the 0, 1, 2... sequence
print(f"Starting rename of {len(files)} files...")

for index, filename in enumerate(files):
    old_path = os.path.join(image_dir, filename)

    # Create the new name based on the index (0.jpg, 1.jpg, etc.)
    new_name = f"{index}.jpg"
    new_path = os.path.join(image_dir, new_name)

    # Rename
    os.rename(old_path, new_path)
    if index < 5: # Print first 5 to verify
        print(f"Renamed: {filename} -> {new_name}")

print("Success! Your images are now 0.jpg, 1.jpg, 2.jpg...")

Starting rename of 159 files...
Renamed: 0.jpg -> 0.jpg
Renamed: 2.jpg -> 1.jpg
Renamed: 3.jpg -> 2.jpg
Renamed: 4.jpg -> 3.jpg
Renamed: 5.jpg -> 4.jpg
Success! Your images are now 0.jpg, 1.jpg, 2.jpg...


In [107]:
import os
import pandas as pd
import re

# Paths
image_dir = "/content/images"
csv_path = "/content/10_Labels.csv"

# 1. Get and Sort files numerically
def extract_number(f):
    s = re.findall(r'\d+', f)
    return int(s[0]) if s else 0

files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
files.sort(key=extract_number)

# 2. Rename files on disk to 0.jpg, 1.jpg, etc.
print(f"Renaming {len(files)} images...")
for index, filename in enumerate(files):
    old_path = os.path.join(image_dir, filename)
    new_name = f"{index}.jpg"
    new_path = os.path.join(image_dir, new_name)
    os.rename(old_path, new_path)

# 3. Update the CSV file
df = pd.read_csv(csv_path)

# Ensure the CSV length matches our image count
# We only take the first 159 rows to match your 159 photos
df = df.iloc[:len(files)].copy()

# Update the 'image' column to match the new names (0.jpg, 1.jpg...)
df['image'] = [f"{i}.jpg" for i in range(len(files))]

# Save the corrected CSV
df.to_csv(csv_path, index=False)

print(f"Success! CSV updated. Total rows: {len(df)}")
print("First 5 rows of updated CSV:")
print(df.head())

Renaming 159 images...
Success! CSV updated. Total rows: 159
First 5 rows of updated CSV:
   Unnamed: 0  image  adv   actual_class
0           0  0.jpg    0  garbage_truck
1           1  1.jpg    1      golf_ball
2           2  2.jpg    1    french_horn
3           3  3.jpg    0      chain_saw
4           4  4.jpg    0      parachute


In [108]:
dataset = AdversarialImageDataset(
    csv_path="/content/10_Labels.csv",
    image_dir="/content/images"
)

# Test the first and last image
img_first, label_first = dataset[0]
img_last, label_last = dataset[158]

print(f"First image shape: {img_first.shape}") # Should be [3, 224, 224]
print(f"Last image (158) loaded successfully!")

First image shape: torch.Size([3, 224, 224])
Last image (158) loaded successfully!


In [109]:
from torch.utils.data import DataLoader, random_split

BATCH_SIZE = 64   # GPU available → increase batch
NUM_WORKERS = 2   # Colab safe

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


In [110]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)


Using device: cpu


In [111]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()


/tmp/ipython-input-178857144.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


In [112]:
from tqdm import tqdm

EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with autocast():  # 🔥 mixed precision
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f}")


Epoch 1/5:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipython-input-453357953.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # 🔥 mixed precision
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1/5: 100%|██████████| 2/2 [00:37<00:00, 18.76s/it]


Epoch 1 | Train Loss: 0.7278


Epoch 2/5: 100%|██████████| 2/2 [00:32<00:00, 16.50s/it]


Epoch 2 | Train Loss: 0.7306


Epoch 3/5: 100%|██████████| 2/2 [00:33<00:00, 16.71s/it]


Epoch 3 | Train Loss: 0.7315


Epoch 4/5: 100%|██████████| 2/2 [00:34<00:00, 17.42s/it]


Epoch 4 | Train Loss: 0.7286


Epoch 5/5: 100%|██████████| 2/2 [00:32<00:00, 16.45s/it]

Epoch 5 | Train Loss: 0.7314


In [114]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

model.eval()

y_true = []
y_pred = []
y_scores = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)

        probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        preds = (probs > 0.5).astype(int)

        y_true.extend(labels.numpy())
        y_pred.extend(preds)
        y_scores.extend(probs)

print("Precision :", round(precision_score(y_true, y_pred), 3))
print("Recall    :", round(recall_score(y_true, y_pred), 3))
print("F1 Score  :", round(f1_score(y_true, y_pred), 3))
print("ROC AUC   :", round(roc_auc_score(y_true, y_scores), 3))


Precision : 0.5
Recall    : 0.467
F1 Score  : 0.483
ROC AUC   : 0.537


In [115]:
import numpy as np
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_true, y_scores)

best_idx = np.argmax(tpr - fpr)   # Youden’s J statistic
best_thresh = thresholds[best_idx]

print("Best threshold:", best_thresh)


Best threshold: 0.35139182


In [116]:
import numpy as np
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_true, y_scores)

best_idx = np.argmax(tpr - fpr)   # Youden’s J statistic
best_thresh = thresholds[best_idx]

print("Best threshold:", best_thresh)


Best threshold: 0.35139182


In [117]:
y_pred = (np.array(y_scores) > best_thresh).astype(int)

print("Precision :", precision_score(y_true, y_pred))
print("Recall    :", recall_score(y_true, y_pred))
print("F1 Score  :", f1_score(y_true, y_pred))


Precision : 0.5454545454545454
Recall    : 0.8
F1 Score  : 0.6486486486486487


In [118]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_true)
weights = compute_class_weight("balanced", classes=classes, y=y_true)
class_weights = torch.tensor(weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)


In [119]:
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [120]:
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=3e-4
)


In [121]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])


In [123]:
torch.save(model.state_dict(), "image_adversarial_cnn.pt")
print("✅ Model saved")


✅ Model saved


In [122]:
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

# same transform used in training
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def test_single_image(image_path, model, device):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        probs = torch.softmax(output, dim=1)[0]

    clean_prob = probs[0].item()
    adv_prob = probs[1].item()

    # 🔥 Decision logic (hackathon-friendly)
    if adv_prob > 0.7:
        action = "BLOCK"
        label = "ADVERSARIAL"
        confidence = adv_prob
    elif adv_prob > 0.4:
        action = "WARNING"
        label = "SUSPICIOUS"
        confidence = adv_prob
    else:
        action = "ALLOW"
        label = "CLEAN"
        confidence = clean_prob

    print("=" * 50)
    print("Image      :", image_path)
    print("Prediction :", label)
    print("Confidence :", f"{confidence*100:.2f}%")
    print("Action     :", action)
    print("=" * 50)

    return label, confidence, action


In [124]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load("image_adversarial_cnn.pt", map_location=device))
model.to(device).eval()

print("Model loaded on:", device)


Model loaded on: cpu


In [125]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load("image_adversarial_cnn.pt", map_location=device))
model.to(device).eval()

print("Model loaded on:", device)


Model loaded on: cpu


In [126]:
from PIL import Image
from torchvision import transforms

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def test_image_colab(image_path):
    image = Image.open(image_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        probs = torch.softmax(model(image), dim=1)[0]

    clean_prob = probs[0].item()
    adv_prob = probs[1].item()

    if adv_prob > 0.7:
        action = "BLOCK"
        label = "ADVERSARIAL"
        conf = adv_prob
    elif adv_prob > 0.4:
        action = "WARNING"
        label = "SUSPICIOUS"
        conf = adv_prob
    else:
        action = "ALLOW"
        label = "CLEAN"
        conf = clean_prob

    print("="*50)
    print("Image:", image_path)
    print("Prediction:", label)
    print("Confidence:", f"{conf*100:.2f}%")
    print("Action:", action)
    print("="*50)


In [127]:
test_image_colab("/content/images/10.jpg")
test_image_colab("/content/images/50.jpg")
test_image_colab("/content/images/120.jpg")


Image: /content/images/10.jpg
Prediction: CLEAN
Confidence: 66.30%
Action: ALLOW
Image: /content/images/50.jpg
Prediction: SUSPICIOUS
Confidence: 43.87%
Action: WARNING
Image: /content/images/120.jpg
Prediction: CLEAN
Confidence: 64.97%
Action: ALLOW


In [128]:
torch.save({
    "model_state": model.state_dict(),
    "model_name": "resnet18",
    "num_classes": 2
}, "image_adversarial_model.pt")

print("Model saved")


Model saved
